In [7]:
import websockets # pip install websocket-client
import ssl
import asyncio
import json
import os
import time

ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
localhost_pem = "cert.pem"
ssl_context.load_verify_locations(localhost_pem)

subj = 1
session = 1
block = 1
headset_info = {} # update this with the headset info


In [12]:
async def send_message(message, websocket):
    attempt = 0
    retries = 3
    responses = []
    finished = False
    while attempt < retries and not finished:
        try:
            message_json = json.dumps(message)
            await websocket.send(message_json)
            while True:
                response = await websocket.recv()
                responses.append(json.loads(response))
                if "warning" in responses[-1]:
                    # print(responses[-1]["warning"]["message"])
                    # Check if the message indicates the end of the process
                    if "The session data has been successfully saved." in responses[-1]["warning"]["message"]["message"]:
                        break  # Exit the loop if the end message is received
                else:
                    print("Received unexpected message:", responses[-1])
                    finished = True
                    break
        except (websockets.exceptions.ConnectionClosedError, websockets.exceptions.WebSocketException) as e:
            attempt += 1
            print(f"Attempt {attempt}: Failed to communicate with WebSocket server - {e}")
            if attempt >= retries:
                print("Maximum retry attempts reached. Stopping.")
                return responses
            await asyncio.sleep(1)  # Wait a bit before retrying
    return responses

async def setup_eeg(websocket):
    # Initialize EEG, e.g., with Emotiv SDK
    # This function needs to be implemented based on your EEG SDK's documentation
    await send_message({
        "id": 1,
        "jsonrpc": "2.0",
        "method": "requestAccess",
        "params": {
            "clientId": os.environ.get('CLIENT_ID'),
            "clientSecret": os.environ.get('CLIENT_SECRET'),
        }
    }, websocket)
    # give it access through launcher
    # refresh the device list
    await send_message({
        "id": 1,
        "jsonrpc": "2.0",
        "method": "controlDevice",
        "params": {
            "command": "refresh"
        }
    }, websocket)
    # query the headsets
    response = await send_message({
        "id": 1,
        "jsonrpc": "2.0",
        "method": "queryHeadsets"
    }, websocket)
    if len(response[-1]["result"]) == 0:
        print("No headsets found")
        exit(1)
    # connect to the headset
    headset = response[-1]["result"][0]["id"] # assuming the first headset, otherwise can manually specifiy
    with open('mapping.json', 'r') as file:
        mapping = json.load(file)
    await send_message({
        "id": 1,
        "jsonrpc": "2.0",
        "method": "controlDevice",
        "params": {
            "command": "connect",
            "headset": headset,
            "mappings": mapping
        }
    }, websocket)
    response = await send_message({ # authorize the connection
        "id": 1,
        "jsonrpc": "2.0",
        "method": "authorize",
        "params": {
            "clientId": os.environ.get('CLIENT_ID'),
            "clientSecret": os.environ.get('CLIENT_SECRET'),
            "debit": 1000
        }
    }, websocket)
    if "error" in response[-1]:
        error = response[-1]["error"]
        print(f"Error in authorizing {error}") # if it gets here, probably didn't set up env variables correctly
        exit(1)
    cortex_token = response[-1]["result"]["cortexToken"]
    # Liscense info
    # response = await send_message({
    #     "id": 1,
    #     "jsonrpc": "2.0",
    #     "method": "getLicenseInfo",
    #     "params": {
    #         "cortexToken": cortex_token
    #     }
    # }, websocket)
    # sometimes requires a delay after authorizing and creating a session
    await asyncio.sleep(0.2)
    response = await send_message({
        "id": 1,
        "jsonrpc": "2.0",
        "method": "createSession",
        "params": {
            "cortexToken": cortex_token,
            "headset": headset,
            "status": "open"
        }
    }, websocket)
    session_id = response[-1]["result"]["id"]
    print("created session", session_id)

    await send_message({
        "id": 1,
        "jsonrpc": "2.0",
        "method": "updateSession",
        "params": {
            "cortexToken": cortex_token,
            "session": session_id,
            "status": "active"
        }
    }, websocket)

    headset_info["headset"] = headset
    headset_info["cortex_token"] = cortex_token
    headset_info["session_id"] = session_id


In [13]:
async with websockets.connect("wss://localhost:6868", ssl=ssl_context) as websocket:
    await setup_eeg(websocket)

    response = await send_message({
        "id": 10,
        "jsonrpc": "2.0",
        "method": "createRecord",
        "params": {
            "cortexToken": headset_info["cortex_token"],
            "session": headset_info["session_id"],
            "title": f"Subject {subj}, Session {session}, Block {block} Recording"
        }
    }, websocket)

    for i in range(10):
        print("marker ", i)
        response = await send_message({
            "id": 1,
            "jsonrpc": "2.0",
            "method": "injectMarker",
            "params": {
                "cortexToken": headset_info["cortex_token"],
                "session": headset_info["session_id"],
                "time": time.time() * 1000,
                "label": "test",
                "value": i
            }
        }, websocket)
        await asyncio.sleep(0.2)

    response = await send_message({
        "id": 20,
        "jsonrpc": "2.0",
        "method": "stopRecord",
        "params": {
            "cortexToken": headset_info["cortex_token"],
            "session": headset_info["session_id"]
        }
    }, websocket)

    recordId = response["result"]["record"]["uuid"]

    response = await send_message({
        "id": 30,
        "jsonrpc": "2.0",
        "method": "exportRecord",
        "params": {
            "cortexToken": headset_info["cortex_token"],
            "folder": "/Users/jonathan/Documents/coding/alljoined/stimulus-emotiv/recordings/test",
            "format": "EDFPLUS",
            "recordIds": [recordId],
            "streamTypes": [
                "EEG",
                "MOTION"
            ]
        }
    }, websocket)

    await asyncio.sleep(5)

    response = await send_message({
        "id": 40,
        "jsonrpc": "2.0",
        "method": "getRecordInfos",
        "params": {
            "cortexToken": headset_info["cortex_token"],
            "recordIds": [recordId]
        }
    }, websocket)

    print("SLEEPYTIME DONE")

    response = await send_message({
        "id": 1,
        "jsonrpc": "2.0",
        "method": "deleteRecord",
        "params": {
            "cortexToken": headset_info["cortex_token"],
            "records": [recordId]
        }
    }, websocket)

    print(response)

    print("BLOCK 10")
    for i in range(10):
        response = await send_message({
            "id": 70,
            "jsonrpc": "2.0",
            "method": "createRecord",
            "params": {
                "cortexToken": headset_info["cortex_token"],
                "session": headset_info["session_id"],
                "title": f"Subject {subj}, Session {session}, Block {block} Recording"
            }
        }, websocket)
        print(i, response)
        await asyncio.sleep(1)


Received unexpected message: {'id': 1, 'jsonrpc': '2.0', 'result': {'accessGranted': True, 'message': 'The user has granted access right to this application.'}}
Received unexpected message: {'id': 1, 'jsonrpc': '2.0', 'result': {'command': 'refresh', 'message': 'Searching for nearby headsets...'}}
Received unexpected message: {'id': 1, 'jsonrpc': '2.0', 'result': [{'connectedBy': 'dongle', 'customName': 'Visual Cortex', 'dfuTypes': ['dfuNone'], 'dongle': '6ff', 'firmware': 'f14', 'firmwareDisplay': '0x0f14', 'headbandPosition': 'back', 'id': 'EPOCFLEX-26FCBE15', 'isDfuMode': False, 'isVirtual': True, 'motionSensors': ['Q0', 'Q1', 'Q2', 'Q3', 'ACCX', 'ACCY', 'ACCZ', 'MAGX', 'MAGY', 'MAGZ'], 'sensors': ['T7', 'TP7', 'CP5', 'CP3', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO9', 'PO7', 'PO3', 'O1', 'O9', 'Pz', 'POz', 'Oz', 'Iz', 'O10', 'O2', 'PO4', 'PO8', 'PO10', 'P10', 'P8', 'P6', 'P4', 'P2', 'CP4', 'CP6', 'TP8', 'T8'], 'settings': {'eegRate': 128, 'eegRes': 14, 'memsRate': 16, 'memsRes': 10, 'mode'

TypeError: list indices must be integers or slices, not str